# TP n°1 du module 6 : Les algorithmes de classification pour le _Machine Learning_

Dans ce TP, nous allons mettre en pratique les principes de l'apprentissage supervisé.

## Objectifs :
- Savoir mettre en place les principaux algorithmes de classification
- Etudier l'impact de leurs paramètres sur leurs performances
- Comparer les performances de différents algorithmes

In [2]:
# Ajoutez ici les imports de librairies nécessaires
#TODO

## Question n°0
Commencez par charger à nouveau le jeu de données Titanic, à partir du csv généré dans le TP1 du module 4.
- Préparez les données d'entraînement et de test qui seront utilisées par la suite.

In [4]:
# Lambda nommée pour afficher un score en pourcentage avec un libellé (avec détail) :
pscore = lambda lib, score, detail='': print(F"{lib}{('',f" ({detail})")[len(str(detail))>0]} : {100*score:.2f}%")

#TODO

## Partie 1 : découvrir Naive Bayes

### Question n°1
Commencez par créer un modèle basé sur Naive Bayes, sans changer les paramètres par défaut, en supposant que la répartition des données correspond à une Gaussienne (loi normale).
- Entraînez-le et testez-le.
- Quelle score (accuracy) obtenez-vous ?
- Que pouvez-vous dire de la précision et du rappel ?
- Comparez avec les scores obtenus sur les arbres de décision au module 5
- Avez-vous des hypothèses pour expliquer cette différence ?

In [7]:
#TODO

#### Observations :
`à compléter`

### Question n°2
Affichez une matrice de corrélation des données du jeu d'entraînement, en y incluant un affichage textuel de la valeur de la corrélation.
- Voyez-vous des informations permettant d'expliquer les performance de l'algorithme _Naive Bayes_ ?

In [10]:
#TODO

#### Observations :
`à compléter`

### Question n°3
Proposez une représentation graphique des attributs continus, permettant de vérifier l'hypothèse que nous avons faite, selon laquelle ces données suivent une loi normale (Gaussienne).

In [13]:
#TODO

#### Observations :
`à compléter`

## Partie 2 : découvrir KNN

### Question n°1
Commencez par créer un modèle KNN, en gardant le nombre de voisins par défaut (à regarder dans la documentation).
- Que pouvez-vous dire de l'accuracy, de la précision et du rappel ?

In [17]:
#TODO

#### Observations :
`à compléter`

### Question n°2
Nous allons maintenant observer l'impact du nombre de voisins à prendre en considération.
- Faite varier k entre 1 et 20.
- Calculez à chaque fois accuracy, précision, et rappel.
- Tracez l'évolution de ces trois scores en fonction de k, sur un même graphique.
- Que constatez-vous ?
- Affichez la valeur de k pour laquelle l'accuracy est la plus élevée.

In [20]:
#TODO

#### Observations :
`à compléter`

### Question n°3
En prenant la valeur de _k_ qui vous semble la plus pertinente, faite varier la dimension (p) utilisée pour calculer la distance de Minkowski entre 2 données.
- Cette distance a-t'elle un fort impact sur les résultats d'accuracy obtenus ?
- Montrez-le en montrant l'évolution de ce score en fonction de _p_<br/> (faire varier entre 1 et 10).
- Ajoutez également la précision et le rappel.

In [23]:
#TODO

#### Observations :
`à compléter`

## Partie 3 : découvrir les SVM

### Question n°1
Créez un modèle de classification basée sur les machines à vecteur de support.
- Dans un premier temps, gardez les options par défaut.
- Que pouvez-vous dire des performances obtenues (accuracy, précision, rappel) ?

In [27]:
#TODO

#### Observations :
`à compléter`

### Question n°2
Testez les différents noyaux disponibles pour l'algorithme SVM (linéaire, polynomial, rbf et sigmoïde).
- Représentez graphiquement l'accuracy, la précision et le rappel, pour chaque noyau.
- Il y en a t'il un qui semble plus pertinent que les autres ?
- Affichez-le, ainsi que les scores obtenus pour ce noyau.

In [30]:
#TODO

### Question n°3
Nous allons essayer d'améliorer les performances obtenues avec le noyau polynomial.
- Utilisez ce noyau (`poly`), et faites varier le degré du polynôme utilisé de 1 à 10.
- Représentez graphiquement l'accuracy, la précision et le rappel, en fonction du degré du polynôme.
- Il y en a-t-il un qui semble plus pertinent que les autres ?
- Affichez-le, ainsi que les scores obtenus pour cette valeur.
- Comparez avec le meilleur score obtenu à la question précédente.

In [32]:
#TODO

## Partie 4 : découvrir les réseaux de neurones (ANN)

### Question n°1
Commençons par étudier le réseau le plus simple : un _perceptron_.
- À l'aide de la classe `sklearn.linear_model.Perceptron`,<br/> créez un perceptron, en gardant les options par défaut.
- Affichez `accuracy`, `précision` et `rappel` : Que pensez-vous de ces performances ?

In [35]:
#TODO

#### Observation :
`à compléter`

### Question n°2
Regardez la documentation pour créer un réseau de neurones (`sklearn.neural_network.MLPClassifier`) :
- Quelle est la structure d'un réseau de neurones par défaut avec scikit-learn ?
- Combien de couches cachées ?
- Combien de neurones par couche ?

_N.B. : Un message d'alerte (⚠Warning: Stochastic Optimizer: Maximum iterations) est suceptible d'apparaître._

#### Réponse :
`à compléter`

### Question n°3
- Créer un réseau de neurones, en gardant ces options par défaut.
- Affichez `accuracy`, `précision` et `rappel` :
    - Que pensez-vous de ces performances, notamment en comparant par rapport au perceptron ?
    - Avez-vous un message d'alerte ?<br/>(⚠Warning: Stochastic Optimizer: Maximum iterations)

In [40]:
#TODO

#### Observation :
`à compléter`

### Question n°4
Si vous avez observé un message d'alerte sur la question précédent :
- Que signifie-t'il selon vous ?
- Que pouvez-vous faire pour y remédier ?
- Proposez un code permettant d'obtenir des résultats, sans message d'alerte.
- Qu'observez-vous sur l'évolution des scores ?

In [43]:
#TODO

#### Observation :
`à compléter`

### Question n°5
Nous allons à présent comparer différentes architectures du réseau de neurones :
- 3 couches de 50 neurones chacune
- 5 couches de 50 neurones chacune
- 3 couches :
    1. 50 neurones,
    2. 100 neurones,
    3. 50 neurones
- 5 couches :
    1. 50 neurones,
    2. 100 neurones,
    3. 50 neurones,
    4. 100 neurones,
    5. 50 neurones

**Les attendus :**
- Représentez graphiquement l'_accuracy_, la _précision_ et le _rappel_, pour chaque architecture.
- Il y en a t'il une qui semble plus pertinente que les autres ?
- Affichez-la, ainsi que les scores obtenus pour cette architecture.
- Comparez avec le score obtenu par l'architecture par défaut.
- Votre code ne doit générer aucun message d'alerte.

In [46]:
#TODO

### Question n°6
En utilisant l'architecture qui vous donnait les meilleures performances, étudier l'impact de la fonction d'activation utilisée sur les performances.
- Représentez sur un graphiques les scores (accuracy, précision et rappel) obtenus pour les quatres fonctions d'activation proposées par _Scikit-Learn_.
- Affichez la fonction qui vous parait la plus pertinente, ainsi que les scores associés.

In [48]:
#TODO

## Partie 5 : comparer les performances des différents algorithmes

Nous allons à présent résumer les différentes performances des algorithmes que vous avez testé dans ce TP :
- Récupérez les meilleurs scores (accuracy) obtenu pour chaque algorithme.
- Représentez-les sur un diagramme en barres, en regroupant par algorithme, et en représentant chaque score par une couleur.
- Un algorithme semble-t'il obtenir de meilleures performances que les autres ?

In [50]:
#TODO

## Partie 6 : optimiser la recherche des paramètres optimaux

Dans ce TP, nous avons souvent cherché à identifier la meilleur combinaison de paramètres. Nous avons procédé par itération, en cherchant à fixer un paramètre avant de faire évoluer les autres. Cette méthode est coûteuse, et pour faire une recherche exhaustive, nécessite, de répéter très souvent le même code. Scikit-learn propose une classe, `sklearn.model_selection.GridSearchCV`, qui va permettre d'optimiser cette recherche de paramétrage optimal.

_Lien vers la documentation :_ [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Le principe est de définir un dictionnaire, où la clé correspond à un paramètre, et la valeur à la liste de valeurs possibles à tester pour le paramètre considéré. 

### Consigne :
Appliquez ce principe pour déterminer la meilleure combinaison possible pour le réseau de neurones, en repartant des différentes configurations testées dans les parties précédentes.

In [53]:
#TODO

In [54]:
#TODO

# Fin du TP !